In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Bidirectional, GRU, Dropout, Dense, Activation
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
csv_path = "aligned_doge.csv"
df = pd.read_csv(csv_path, parse_dates=['dateTime'])
df = df.sort_values('dateTime')

In [ ]:
scaler = MinMaxScaler()

scaled_close = scaler.fit_transform(df[["close", "Sentiment"]])


In [ ]:
scaled_close = scaled_close[~np.isnan(scaled_close).any(axis=1)]
scaled_close = scaled_close.reshape(-1, 2)

In [ ]:
SEQ_LEN = 100

def to_sequences(data, seq_len):
    d = []

    for index in range(len(data) - seq_len):
        d.append(data[index: index + seq_len])

    return np.array(d)

def preprocess(data_raw, seq_len, train_split):

    data = to_sequences(data_raw, seq_len)

    num_train = int(train_split * data.shape[0])

    X_train = data[:num_train, :-1, :]
    y_train = data[:num_train, -1, 0]

    X_test = data[num_train:, :-1, :]
    y_test = data[num_train:, -1, 0]

    return X_train, y_train, X_test, y_test


X_train, y_train, X_test, y_test =\
 preprocess(scaled_close, SEQ_LEN, train_split = 0.95)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
DROPOUT = 0.2
WINDOW_SIZE = SEQ_LEN - 1

model = keras.Sequential()

model.add(Bidirectional(
  GRU(WINDOW_SIZE, return_sequences=True),
  input_shape=(WINDOW_SIZE, X_train.shape[-1])
))
model.add(Dropout(rate=DROPOUT))

model.add(Bidirectional(
  GRU((WINDOW_SIZE * 2), return_sequences=True)
))
model.add(Dropout(rate=DROPOUT))

model.add(Bidirectional(
  GRU(WINDOW_SIZE, return_sequences=False)
))

model.add(Dense(units=1))

model.add(Activation('linear'))

In [ ]:
BATCH_SIZE = 64

model.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['accuracy']
)

history = model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=BATCH_SIZE,
    shuffle=False,
    validation_split=0.1
)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rc

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['test', 'train'], loc='upper left')
plt.show()

In [ ]:
y_hat = model.predict(X_test)

# y_test_inverse = scaler.inverse_transform(y_test)
# y_hat_inverse = scaler.inverse_transform(y_hat)
 
plt.plot(y_test, label="Actual Price", color='green')
plt.plot(y_hat, label="Predicted Price", color='red')
 
plt.title('Price prediction')
plt.xlabel('Time [days]')
plt.ylabel('Price')
plt.legend(loc='best')
 
plt.show()